In [1]:
import numpy as np
import math
import scipy.linalg as la

# Semana 2 PIBIC

## Gerando Tabelas Verdade (Truth Tables)

Para a geração de tabelas verdade com n inputs no circuito, com $n \ge 1$. Esta é uma função recursiva que gera uma tabela verdade em forma de matriz para n inputs arbitrários.

Talvez existam formas mais rápidas e menores de fazê-las, mas esta função é mais legível para os usuários e para os parceiros da pesquisa. Modifiquei o nome das variáveis para auxiliar no processo.

Além disso, modifiquei-a para que retornasse um numpy array ao invés de um array convencional do python, aumentando a eficiência da função visto que numpy arrays são mais rápidos de se fazer modificações na função e até mesmo posteriores.

In [34]:
def Tv(NumberOfInputs):
    """
    Retorna uma tabela verdade (TruthTable) dado um circuito com n inputs maiores ou iguais a 1
    Checa se a variável NumberOfInputs é inteira. Se sim, continua normalmente, se não lança uma exceção com a mensagem
    'Parameter \'NumberOfInputs\' must be a integer'

    Parâmetros
    ------------------------------------------------
    :param NumberOfInputs: número de inputs do circuito lógico, deve ser um número inteiro (obrigatoriamente)
    
    Retorno
    ------------------------------------------------
    :return: Retorna a tabela verdade como uma lista do Python (caso necessário, transformar o retorno em numpy array)
    """
    if not isinstance(NumberOfInputs, int):
        raise ValueError('Parameter \'NumerOfInputs\' must be a integer')
    if NumberOfInputs < 1:
        return [[]]
    
    SubTable = Tv(NumberOfInputs - 1)
    return [row + [v] for row in SubTable for v in [0,1]]

In [35]:
Table = np.array(Tv(3), dtype=float)
Table

array([[0., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 1.],
       [1., 0., 0.],
       [1., 0., 1.],
       [1., 1., 0.],
       [1., 1., 1.]])

A única verificação necessária nessa função se dá no momento da passagem do argumento $NumberOfInputs$. Se o número passado não for um inteiro, a função retorna um erro e não continua seus cálculos.

Além disso, caso o argumento de número de inputs for menor ou igual a 0, a função retorna um numpy array vazio.

## Modulação PAM

Quando tratamos de tabelas verdade de circuitos eletrônicos, muitas vezes precisamos converter esses sinais em Modulação por Amplitude do Pulso (PAM: Pulse Amplitude Moduling) para realizarmos cálculos ou verificações e depois retornamos a representação do circuito a meneira original, com 0's e 1's.

Para isso, convertemos os campos (labels) das tabelas verdades por $1+\epsilon$ caso o valor no campo seja 1 ou $1-\epsilon$ caso seja 0. A matriz resultante é chamada de $\textbf{Imp}$.

No exemplo a seguir, utilizaremos a variável table definida no código acima para os testes e o valor $\epsilon = 5$:

In [39]:
def PAM(TruthTable, epsilon):
    """
    Função que retorna uma modulação PAM para uma dada tabela verdade (TruthTable) e certo epsilon real arbitrário.
    Não modifica a tabela verdade original, apenas realiza alterações em uma cópia e a retorna no final.
    Funciona para qualquer tabela verdade (TruthTable) com n inputs (Matriz não precisa ser quadrada)

    Parâmetros
    ------------------------------------------------
    :param TruthTable: tabela verdade a ser analisada e transformada em modulação PAM
    :param epsilon: epsilon no qual os valores serão calculados
    
    Retorno
    ------------------------------------------------
    :return: retorna um numpy array como matriz da modulação PAM da tabela verdade recebida como parâmetro
    """
    if epsilon <= 0:
        raise ValueError(f'Epsilon must be major than 0')
    PamTable = TruthTable.copy()
    rows, columns = PamTable.shape
    for row in range(rows):
        for column in range(columns):
            if PamTable[row][column] == 1:
                PamTable[row][column] = 1 + epsilon
            else:
                PamTable[row][column] = 1 - epsilon
                # Colocar ValueError valores diferentes de 0 ou 1
                # Fazer numpy array receber valores float
    return PamTable

In [40]:
Inp = PAM(Table, 1.4)
print(f'INP: \n{Inp}\n\nTABELA VERDADE ORIGINAL: \n{Table}')

INP: 
[[-0.4 -0.4 -0.4]
 [-0.4 -0.4  2.4]
 [-0.4  2.4 -0.4]
 [-0.4  2.4  2.4]
 [ 2.4 -0.4 -0.4]
 [ 2.4 -0.4  2.4]
 [ 2.4  2.4 -0.4]
 [ 2.4  2.4  2.4]]

TABELA VERDADE ORIGINAL: 
[[0. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 1.]
 [1. 0. 0.]
 [1. 0. 1.]
 [1. 1. 0.]
 [1. 1. 1.]]


In [37]:
def InvPAM(PAMTable):
    """
    Função que retorna uma nova tabela verdade com modulação bniária (processo inverso da modulação PAM)
    
    Parâmetros
    ------------------------------
    :param PAMTable: Tabela verdade em modulação PAM (1 +- epsilon)
    
    Retorno
    ------------------------------
    :return: Retorna uma nova matriz como numpy array que representa a tabela verdade original (inversa da PAM)
    """
    rows, columns = PAMTable.shape
    ResultTable = np.ones(shape=(rows, columns), dtype=float)
    for row in range(rows):
        for column in range(columns):
            if PAMTable[row][column] < 1:
                ResultTable[row][column] = 0
    return ResultTable

In [38]:
InvPam = InvPAM(Inp)
print(f'Imp: \n\n{Inp}\nInvPAM: \n\n{InvPam}')

Imp: 

[[-0.4 -0.4 -0.4]
 [-0.4 -0.4  2.4]
 [-0.4  2.4 -0.4]
 [-0.4  2.4  2.4]
 [ 2.4 -0.4 -0.4]
 [ 2.4 -0.4  2.4]
 [ 2.4  2.4 -0.4]
 [ 2.4  2.4  2.4]]
InvPAM: 

[[0. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 1.]
 [1. 0. 0.]
 [1. 0. 1.]
 [1. 1. 0.]
 [1. 1. 1.]]


O único erro que a função PAM poderia dar era em retornar uma tabela verdade modulada erroneamente, o que não ocorre devido ao retorno da função $InvPAM$ ser exatamente igual à tabela verdade original. Assim, nenhum erro é cometido pelas funções.

## Geração de Tabelas Verdade Padrões (OR, AND)

Para dar continuidade à pesquisa, precisamos de uma função que nos retorne tabelas verdade de operadores lógicos padrões (OR, AND).

In [41]:
def OrTruthTableGenerator(TruthTable):
    """
    Retorna uma tabela verdade do circuito lógico OR de acordo com uma tabela verdade (TruthTable). 
    Checa se a tabela está vazia, se sim a retorna (vazia), se não, realiza os cálculos.
    
    Parâmetros
    ------------------------------------------------
    :param TruthTable: tabela verdade a ser analisada
    
    Retorno
    ------------------------------------------------
    :return: Retorna a coluna (numpt array) com os valores da tabela verdade OR dos inputs da tabela verdade recebida 
    como parâmetro
    """
    rows, columns = TruthTable.shape
    ResultTable = np.zeros(shape=(rows, 1), dtype=float)
    
    if TruthTable.size == 0:
        return TruthTable
    
    for row in range(rows):
        BooleanRowArray = TruthTable[row][:]
        if np.any(BooleanRowArray):
            ResultTable[row][0] = 1
        
    return ResultTable

In [42]:
def AndTruthTableGenerator(TruthTable):
    """
    Retorna uma tabela verdade do circuito lógico AND de acordo com uma tabela verdade (TruthTable). 
    Checa se a tabela está vazia, se sim a retorna (vazia), se não, realiza os cálculos.
    
    Parâmetros
    ------------------------------------------------
    :param TruthTable: tabela verdade a ser analisada
    
    Retorno
    ------------------------------------------------
    :return: Retorna a coluna (numpy array) com os valores da tabela verdade AND dos inputs da tabela verdade recebida 
    como parâmetro
    """
    rows, columns = TruthTable.shape
    ResultTable = np.zeros(shape=(rows, 1), dtype=float)
    
    if TruthTable.size == 0:
        return TruthTable
    
    for row in range(rows):
        BooleanRowArray = TruthTable[row][:]
        if np.all(BooleanRowArray):
            ResultTable[row][0] = 1;
    
    return ResultTable

In [43]:
OR = OrTruthTableGenerator(Table)
AND = AndTruthTableGenerator(Table)
print(f'OR TRUTH TABLE: \n{OR} \n\nAND TRUTH TABLE: \n{AND}')

OR TRUTH TABLE: 
[[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]] 

AND TRUTH TABLE: 
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]


Perceba que os exemplos utilizados para os cálculos das funções são feitos utilizando tabelas verdade de 3 inputs. No entanto, como já documentado, essas funções aceitam tabelas verdade de n inputs, sendo n um número natural.

## Código de Teste

Os códigos abaixo realizam testes com as funções abordadas, seguindo o modelo (exemplo para o arranjo triangular):

$A = ArranjoT rinagular(α, β, γ) \newline T = MatrizTransferencia(A) \newline Inp = PAM(TV(3)) \newline Out = T Inp^{T} \newline Out = Out^{T} \newline OutPAM = PAM(OR)$

In [44]:
def ArranjoTriangular(alpha, beta, gama):
    A = np.array([[0, complex(0, gama), complex(0, beta)], 
                  [complex(0, gama), 0, complex(0, alpha)], 
                  [complex(0, beta), complex(0, alpha), 0]])
    return A

In [45]:
def MatrizTransferenciaTriangular(A):
    """
    Retorna a matriz de transferência triangular T a partir de uma matriz de arranjo triangular A a partir dos passos já citados
    Erro percentual de 10^(-14) a 10^(-16), ínfimo.
    
    Parâmetros
    --------------------------------------
    :param A: matriz de Arranjo Triangular como numpy array
    
    Retorno
    
    :return:  matriz numpy array de transferência triangular T calculada com erro ínfimo.
    """
    eigvals, eigvecs = la.eig(A)
    D = np.array([[eigvals[0], 0, 0], 
                  [0, eigvals[1], 0], 
                  [0, 0, eigvals[2]]])
    P = eigvecs.copy()
    E = np.array([[np.exp(eigvals[0]), 0, 0], 
                  [0, np.exp(eigvals[1]), 0], 
                  [0, 0, np.exp(eigvals[2])]])
    
    if np.allclose(A, P @ D @ la.inv(P), atol=1e-17):
        raise ValueError(f'Parameter Matrix A has not property A == PDP^(-1) for a tolerance of {1e-17}')
    
    # T é a matriz de transferência
    T = P @ E @ la.inv(P)
    
    return T

In [57]:
A = ArranjoTriangular(1, 2, 3)
T = MatrizTransferenciaTriangular(A)
Inp = PAM(np.array(Tv(3), dtype=float), 0.5)

Out = T @ Inp.T
Out = Out.T
OutPAM = PAM(OR, 5)

In [58]:
print(f'A: \n{A}\n\nT: \n{T}\n\nInp: \n{Inp}\n\nOut: \n{Out}\n\nOutPAM: \n{OutPAM}')

A: 
[[0.+0.j 0.+3.j 0.+2.j]
 [0.+3.j 0.+0.j 0.+1.j]
 [0.+2.j 0.+1.j 0.+0.j]]

T: 
[[-0.78640394-0.33599426j  0.27205028-0.3998916j  -0.03223645-0.18361063j]
 [ 0.27205028-0.3998916j  -0.40191798-0.48536506j -0.59284716+0.13225087j]
 [-0.03223645-0.18361063j -0.59284716+0.13225087j  0.23889194-0.73431637j]]

Inp: 
[[0.5 0.5 0.5]
 [0.5 0.5 1.5]
 [0.5 1.5 0.5]
 [0.5 1.5 1.5]
 [1.5 0.5 0.5]
 [1.5 0.5 1.5]
 [1.5 1.5 0.5]
 [1.5 1.5 1.5]]

Out: 
[[-2.73295055e-01-0.45974824j -3.61357431e-01-0.37650289j
  -1.93095836e-01-0.39283806j]
 [-3.05531509e-01-0.64335887j -9.54204593e-01-0.24425202j
   4.57961088e-02-1.12715444j]
 [-1.24477226e-03-0.85963984j -7.63275414e-01-0.86186795j
  -7.85942997e-01-0.26058719j]
 [-3.34812261e-02-1.04325047j -1.35612258e+00-0.72961707j
  -5.47051053e-01-0.99490356j]
 [-1.05969900e+00-0.79574251j -8.93071480e-02-0.77639449j
  -2.25332289e-01-0.57644869j]
 [-1.09193545e+00-0.97935313j -6.82154310e-01-0.64414361j
   1.35596549e-02-1.31076506j]
 [-7.87648712e-01-1.195

## Testes

### 1) A modulação PAM e a função InvPAM funcionam corretamente? (SIM)

In [15]:
Original = np.array(Tv(3))
Inp = PAM(Original, 5)
RetornoInv = InvPAM(Inp)
print(f'Original: \n\n{Original}\nINP: \n\n{Inp}\n\nRetorno InvPAM: \n\n{RetornoInv}')
print(np.allclose(Original, RetornoInv))

Original: 

[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]
INP: 

[[-4 -4 -4]
 [-4 -4  6]
 [-4  6 -4]
 [-4  6  6]
 [ 6 -4 -4]
 [ 6 -4  6]
 [ 6  6 -4]
 [ 6  6  6]]

Retorno InvPAM: 

[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]
True


### 2) O cálculo de Out functiona corretamente? Análise do erro (SIM)

Para o cálculo do erro da matriz $Out$, faremos manipulações de matrizes assim como fizemos com a matriz $E$ no estágio da semana anterior. Seja $Out = T Inp^T$. Podemos comparar visualmente as duas matrizes, ou podemos utilizar o método ``np.allclose(atol=1e-18)``, na qual atol é uma tolerância ínfima, quase nula, mas mesmo assim vemos que a função retorna True.

Ou seja, os cálculos são feitos de maneira correta.

In [16]:
print(f'out: \n\n{Out.T}\n\nT @ Inp.T: \n\n{T @ Inp.T}')

out: 

[[ 2.18636044+3.67798595j  1.8639959 +1.84187969j  4.90686327-0.32093003j
   4.58449873-2.15703629j -5.67767896+0.31804331j -6.00004349-1.51806295j
  -2.95717613-3.68087267j -3.27954066-5.51697893j]
 [ 2.89085945+3.01202312j -3.03761217+4.33453186j -1.12832038-1.84162744j
  -7.056792  -0.5191187j   5.61136228-0.98689286j -0.31710934+0.33561588j
   1.59218245-5.84054342j -4.33628917-4.51803468j]
 [ 1.54476669+3.1427045j   3.93368613-4.20045924j -4.38370493+4.46521324j
  -1.99478549-2.8779505j   1.22240215+1.30659825j  3.61132159-6.03656549j
  -4.70606947+2.62910698j -2.31715003-4.71405676j]]

T @ Inp.T: 

[[ 2.18636044+3.67798595j  1.8639959 +1.84187969j  4.90686327-0.32093003j
   4.58449873-2.15703629j -5.67767896+0.31804331j -6.00004349-1.51806295j
  -2.95717613-3.68087267j -3.27954066-5.51697893j]
 [ 2.89085945+3.01202312j -3.03761217+4.33453186j -1.12832038-1.84162744j
  -7.056792  -0.5191187j   5.61136228-0.98689286j -0.31710934+0.33561588j
   1.59218245-5.84054342j -4.33628

In [17]:
np.allclose(Out.T, T @ Inp.T, atol=1e-18)

True